In [29]:
# Importación de librerias
import pandas as pd
import ast
import gzip
from datetime import datetime
import gzip


In [30]:
# Carga del dataframe con el archivo steam_games.json.gz
df_reviews = pd.read_parquet('Dataset/df_user_reviews_final.parquet') 
df_reviews.head(3)

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews,reviews_date,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,2
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013-10-14,2


In [31]:
# Revisamos las coolumnas
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'reviews', 'reviews_date', 'sentiment_analysis'],
      dtype='object')

In [32]:
# Eliminamos las columnas que no son necesarias
df_reviews.drop(columns=['user_id', 'user_url', 'reviews_helpful', 'reviews', ], inplace=True)
df_reviews.head()


,reviews_item_id,reviews_recommend,reviews_date,sentiment_analysis
0,1250,True,2011-11-05,2
1,251610,True,2014-06-24,2
3,250320,True,2013-10-14,2
4,211420,True,2014-04-15,1
5,249130,True,2014-05-05,0


In [33]:
#Verificamos el estado de nulos
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48909 entries, 0 to 232127
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   reviews_item_id     48909 non-null  object
 1   reviews_recommend   48909 non-null  bool  
 2   reviews_date        48909 non-null  object
 3   sentiment_analysis  48909 non-null  int64 
dtypes: bool(1), int64(1), object(2)
memory usage: 1.5+ MB


In [34]:
# Revisar la columna release_date si tiene valore nulos
print(df_reviews['reviews_date'].isna().sum())
#print(df_steam_games['release_date'].isna())
df_reviews.isna().sum()

0


reviews_item_id       0
reviews_recommend     0
reviews_date          0
sentiment_analysis    0
dtype: int64

In [35]:
# Convertir la columna release_date a datetime
df_reviews['reviews_date'] = pd.to_datetime(df_reviews['reviews_date'], errors='coerce')
df_reviews.head()

,reviews_item_id,reviews_recommend,reviews_date,sentiment_analysis
0,1250,True,2011-11-05,2
1,251610,True,2014-06-24,2
3,250320,True,2013-10-14,2
4,211420,True,2014-04-15,1
5,249130,True,2014-05-05,0


In [36]:
# Adicionar una columna 'year' que extrae el año de la columna 'release_date'
df_reviews['year'] = df_reviews['reviews_date'].dt.year
df_reviews.head()

,reviews_item_id,reviews_recommend,reviews_date,sentiment_analysis,year
0,1250,True,2011-11-05,2,2011
1,251610,True,2014-06-24,2,2014
3,250320,True,2013-10-14,2,2013
4,211420,True,2014-04-15,1,2014
5,249130,True,2014-05-05,0,2014


In [37]:
df_reviews.drop(columns=['reviews_date'], inplace=True)
df_reviews.head()

,reviews_item_id,reviews_recommend,sentiment_analysis,year
0,1250,True,2,2011
1,251610,True,2,2014
3,250320,True,2,2013
4,211420,True,1,2014
5,249130,True,0,2014


In [38]:
# Carga del dataframe con el archivo steam_games.json.gz
df_steam_games = pd.read_json('dataset/steam_games.json.gz',lines=True,compression='gzip')
df_steam_games.drop(columns=['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',  'tags', 'reviews_url', 'specs', 'price', 'early_access'], inplace=True)
df_steam_games = df_steam_games.dropna(how='all')
df_steam_games.head(3)

,id,developer
88310,761140.0,Kotoshiro
88311,643980.0,Secret Level SRL
88312,670290.0,Poolians.com


In [39]:
# Revisar la columna release_date si tiene valore nulos
print(df_steam_games['id'].isna().sum())
#print(df_steam_games['release_date'].isna())
df_steam_games.isna().sum()

1


id              1
developer    3298
dtype: int64

In [40]:
# Eliminamos los registros que contiene datos nulos
df_steam_games = df_steam_games.dropna()

# Verificar si hay valores nulos en el dataframe
null_counts = df_steam_games.isnull().sum()

# Imprimir el conteo de valores nulos por columna
print(null_counts)

id           0
developer    0
dtype: int64


In [41]:
# Convertir las columnas de unión al mismo tipo de datos
df_reviews['reviews_item_id'] = df_reviews['reviews_item_id'].astype(float)
df_steam_games['id'] = df_steam_games['id'].astype(float)

# Verificar que las conversiones fueron exitosas
print(df_reviews['reviews_item_id'].dtype)
print(df_steam_games['id'].dtype)

float64
float64


In [42]:
 # Unir con el dataset de juegos para obtener el desarrollador
merged_df = df_reviews.merge( df_steam_games, left_on='reviews_item_id', right_on='id')
merged_df.head()

,reviews_item_id,reviews_recommend,sentiment_analysis,year,id,developer
0,1250.0,True,2,2011,1250.0,Tripwire Interactive
1,250320.0,True,2,2013,250320.0,Telltale Games
2,211420.0,True,1,2014,211420.0,FromSoftware
3,249130.0,True,0,2014,249130.0,Traveller's Tales
4,4000.0,True,1,2013,4000.0,Facepunch Studios


In [43]:
# Eliminamos una de las columnas id
merged_df.drop(columns=['reviews_item_id','sentiment_analysis'], inplace=True)
merged_df.head()

,reviews_recommend,year,id,developer
0,True,2011,1250.0,Tripwire Interactive
1,True,2013,250320.0,Telltale Games
2,True,2014,211420.0,FromSoftware
3,True,2014,249130.0,Traveller's Tales
4,True,2013,4000.0,Facepunch Studios


In [44]:
# Revisamos la estructura de la data
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41378 entries, 0 to 41377
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   reviews_recommend  41378 non-null  bool   
 1   year               41378 non-null  int32  
 2   id                 41378 non-null  float64
 3   developer          41378 non-null  object 
dtypes: bool(1), float64(1), int32(1), object(1)
memory usage: 848.7+ KB


In [45]:
# Almacenar este dataset resultante para la ejección del cuarto EndPoint
merged_df.to_parquet('dataset/df_best_developer_year.parquet', engine='pyarrow')

In [46]:
# Se carga el dataset para la ejecución del EndPont 
merged_df = pd.read_parquet('Dataset/df_best_developer_year.parquet') 
merged_df.head()

,reviews_recommend,year,id,developer
0,True,2011,1250.0,Tripwire Interactive
1,True,2013,250320.0,Telltale Games
2,True,2014,211420.0,FromSoftware
3,True,2014,249130.0,Traveller's Tales
4,True,2013,4000.0,Facepunch Studios


In [47]:
# Revisamos las columnas
merged_df.columns

Index(['reviews_recommend', 'year', 'id', 'developer'], dtype='object')

In [53]:
# Elaboramos la base de la funcion API

anio=2013

# Filtrar las filas donde 'year' es igual al año dado y 'reviews_recommend' es True
filtered_df = merged_df[(merged_df['year'] == anio) & (merged_df['reviews_recommend'] == True)]

# Contar recomendaciones por desarrollador
developer_counts = filtered_df['developer'].value_counts()

# Obtener los top 3 desarrolladores
top_3_developers = developer_counts.head(3)

# Formatear el resultado como un diccionario
result = {f"Puesto {i+1}": developer for i, developer in enumerate(top_3_developers.index)}      

print(result)

{'Puesto 1': 'Valve', 'Puesto 2': 'Facepunch Studios', 'Puesto 3': 'Re-Logic'}
